1. [Useful Functions](#1\)-Useful-Functions)
2. [Loading information](#2\)-Loading-information)
3. [Colour-based segmentation using HSV colour space](#3\)-Colour-based-segmentation-using-HSV-colour-space)
4. [Shape-based detection based on contour area](#4\)-Shape-based-detection-based-on-contour-area)
5. [Shape-based detection based on contour perimeter](#5\)-Shape-based-detection-based-on-contour-perimeter)
6. [Compare and select the optimal bounding box for each image](#6\)-Compare-and-select-the-optimal-bounding-box-for-each-image)

In [ ]:
import os
import sys
# Python 3.7 is required
assert sys.version_info >= (3,7)
import cv2 as cv
import numpy as np
import pandas as pd
import math
import copy
from IPython.display import display
import matplotlib.pyplot as plt

# Make sure that optimization is enabled
if not cv.useOptimized():
    cv.setUseOptimized(True)

cv.useOptimized()

# 1) Useful Functions

In [ ]:
#1) function to load all images from specified folder and return image and image name list
def load(folder):
    cnt=0
    imgs = []
    imgsName = []
    for filename in os.listdir(folder):
        img = cv.imread(os.path.join(folder,filename))
        img=cv.cvtColor(img, cv.COLOR_BGR2RGB)
        if img is not None: 
            imgs.append(img)
            imgsName.append(filename)
    imgs = np.array(imgs, dtype=object)
    imgsName = np.array(imgsName)
    return imgs, imgsName

#2) function to show images in columns of 5 based on some image list input
def show(imgs):
    #define number of images and the number of columns and rows of images required to show all images
    n=len(imgs)
    col=5
    row=(n+col-1)//col 

    #define figure size
    fig=plt.figure(figsize=(100//col,100))

    #display all images from the images list
    for i in range(n):
        ax=fig.add_subplot(row,col,i+1)
        plt.imshow(imgs[i],cmap='gray'), plt.axis('off'), plt.title("image {}: {}".format(i,imgsName[i]))
    plt.show()

#3) function to return area of a mask
def area(image):
    gray = cv.cvtColor(image, cv.COLOR_BGR2GRAY)
    thresh = cv.threshold(gray,0,255,cv.THRESH_OTSU + cv.THRESH_BINARY)[1]
    pixels = cv.countNonZero(thresh)
    return pixels

#4) function to return a average bounding box accuracy, precision, recall and f1 and prints each image accuracy, precision, recall and f1 
#based on a 2d numpy array input in a specified format of each row representing an image having format [index, x1, y1, x2, y2]
def score(ans, csv=False):
    #define final_ans list that stores [accuracy, precision, recall and f1] of each image (will be appended to later)
    final_ans = []

    #for each image's bounding box information
    for i in ans:
        #get predicted bounding box information
        index, x1, y1, x2, y2 = i
        
        #get ground truth bounding box information
        img = annotdict[imgsName[index]]
        width, height, samplex1, sampley1, samplex2, sampley2 = img
        
        #create a blank image with the same dimension of the current image for predicted and ground truth mask
        pred_mask = np.zeros((height,width,3), np.uint8)
        sample_mask = np.zeros((height,width,3), np.uint8)
        
        #create the mask for sample and ground truth 
        cv.rectangle(sample_mask, (samplex1,sampley1), (samplex2, sampley2), (255, 255, 255), -1)
        cv.rectangle(pred_mask, (x1,y1), (x2, y2), (255, 255, 255), -1)

        #generate tp, tn, fp and fn masks based on predicted and ground truth mask
        tp_mask = np.bitwise_and(pred_mask,sample_mask)
        tn_mask = np.bitwise_not(np.bitwise_or(pred_mask,sample_mask))
        fp_mask = np.bitwise_and(pred_mask,np.bitwise_not(sample_mask))
        fn_mask = np.bitwise_and(sample_mask,np.bitwise_not(pred_mask))

        #calculate area of each mask
        tp = area(tp_mask)
        tn = area(tn_mask)
        fp = area(fp_mask)
        fn = area(fn_mask)
        
        #use area of tp, tn, fp and fn to determine accuracy, precision, recall and f1 score
        accuracy = (tp+tn)/(tp+tn+fp+fn)
        precision = tp/(fp+tp)
        recall = tp/(fn+tp)
        f1 = tp/(tp+(fn+fp)/2)
        
        #append [accuracy,precision,recall,f1] of current image to final_ans 
        final_ans.append([accuracy,precision,recall,f1])
        
    #convert final ans to pandas dataframe for nicer looking output
    df = pd.DataFrame(final_ans)
    df.columns = ['accuracy', 'precision', 'recall', 'f1']
    pd.set_option('display.max_rows', None)
    pd.set_option('display.max_columns', None)
    pd.set_option('display.width', None)
    pd.set_option('display.max_colwidth', None)
    
    #print statistics of each column
    print(df.describe())
    
    #display the dataframe with style
    display(df.style.applymap(clr))
    print()

    #if csv is set to true, create csv file with the accuracy, precision, recall and f1 score of each image(used to make table for report)
    if csv == True:
        df.to_csv('ans.csv', index=False)
    
    #return mean accuracy, precision, recall and f1
    return df['accuracy'].mean(), df['precision'].mean(), df['recall'].mean(), df['f1'].mean()

#5) function for setting background-colour when displaying dataframe of each image accuracy, precision, recall and f1 result
def clr(val):
    #if value=0, red background
    if math.isclose(val,0,abs_tol=1e-6):
        color = 'red'
    #else if value<0.6, light pink background
    elif val < 0.6:
        color = 'lightpink'
    #else if value > 0.8, palegreen background
    elif val > 0.8:
        color = 'palegreen'
    #no change in background colour otherwise
    else:
        color = ''
    return 'background-color: %s' % color

#6) sort function for sorting based on increasing aspect ratio to select the best bounding box result of an image from a list of options
def sort_func(k):
    #in actuality this is a function to return a modified aspect ratio that is larger than 1(we return the inverse if aspect ratio is less than 1)
    #for the purpose of easier sorting to get bounding box that is the closest to aspect ratio = 1
    if (k[4]-k[2]) >= (k[3]-k[1]):
        return (k[4]-k[2])/(k[3]-k[1])
    else:
        return (k[3]-k[1])/(k[4]-k[2])

# 2) Loading information

In [ ]:
#annotation dictionary that stores image name as key and image information as value in the form of [width, height, x1, y1, x2, y2] (will be appended later)
annotdict = {}

#populates annotdict using the annotation text file
with open("TsignRecgTrain4170Annotation.txt") as annotations:
    for line in annotations:
        arr = line.split(';')
        key = arr[0]
        for i in range(1,8): arr[i]=int(arr[i])
        annotdict[key] = arr[1:7]

#load all images and their file names from specified folder
imgs, imgsName = load("sign")

#show loaded images
show(imgs) 

# 3) Colour-based segmentation using HSV colour space

In [ ]:
#deep copy original image list
imgs_copy = copy.deepcopy(imgs)
count = 0;

#produce answer list to store bounding box information(will be appended later)
hsv_ans = []

for img in imgs_copy:
    #convert to hsv
    img_hsv = cv.cvtColor(img, cv.COLOR_RGB2HSV)

    # Create red mask
    red_low = (160,50,50)
    red_high = (180,255,255)

    mask_red = cv.inRange(img_hsv, red_low ,red_high)

    # Create blue mask
    blue_low = (0,150,20)
    blue_high = (120,255,255)

    mask_blue = cv.inRange(img_hsv, blue_low, blue_high)

    # Create yellow mask
    yellow_low = (15,60,180)
    yellow_high = (35,255,255)

    mask_yellow = cv.inRange(img_hsv, yellow_low, yellow_high)

    # Combine all masks
    final_mask = cv.add(mask_red, cv.add(mask_blue, mask_yellow))

    # Segment regions with blue, red, or yellow colour
    final_result = cv.bitwise_and(img, img, mask=final_mask)

    # clean up the segmentation using blur filter
    blur = cv.GaussianBlur(final_result, (3, 3), 0)

    #canny edge detection
    canny = cv.Canny(img, 50, 250, 3)

    #find contours
    contours, hierarchy = cv.findContours(canny, cv.RETR_EXTERNAL,cv.CHAIN_APPROX_SIMPLE)

    #if contour exist, find contour with largest area and produce bounding box information
    if(len(contours)>=1):
        #maxVal = cv.arcLength(contours[0],True)
        maxVal = cv.contourArea(contours[0])
        index=0

        for i in range(len(contours)):
            #metric = cv.arcLength(contours[i],True)
            metric = cv.contourArea(contours[i])
            if(metric>=maxVal):
                index=i
                maxVal = metric

        cnt = contours[index]
        x, y, w, h = cv.boundingRect(cnt)
    #else if no contours detected, produce error message and produce bounding box information with the entire image as the bounding box    
    else: 
        print("no contour detected for image",count)
        x = 0
        y = 0
        w, h = annotdict[imgsName[count]][:2]
        
    #produce bounding box with thickness that scales off the size of the image(for consistency)
    imgWidth, imgHeight = annotdict[imgsName[count]][:2]
    thickness = math.ceil(max(1,0.02*min(imgWidth, imgHeight)))
    cv.rectangle(img, (x,y), (x+w, y+h), (0, 255, 0), int(thickness))
    
    #append bounding box information into hsv_ans answer list
    hsv_ans.append([count,x,y,x+w,y+h])
    count+=1

#evaluate score of hsv_ans_list
hsv_ans=np.array(hsv_ans)
a, p, r, f = score(hsv_ans)

#show image with bounding box
show(imgs_copy)

# 4) Shape-based detection based on contour area

In [ ]:
#deep copy original image list
imgs_copy = copy.deepcopy(imgs)
count = 0

#produce answer list to store bounding box information(will be appended later)
contour_area_ans = []

for img in imgs_copy:
    img=cv.medianBlur(img,5,img)
    img_copy = cv.bilateralFilter(img,7,29,3)

    #canny edge detection
    canny = cv.Canny(img_copy, 20, 50, 3)

    #find contours
    contours, hierarchy = cv.findContours(canny, cv.RETR_EXTERNAL,cv.CHAIN_APPROX_SIMPLE)
    
    #if contour exist, find contour with largest area and produce bounding box information
    if(len(contours)>=1):
        maxVal = cv.contourArea(contours[0])
        index=0
        for i in range(len(contours)):
            metric = cv.contourArea(contours[i])
            if(metric>=maxVal):
                index=i
                maxVal = metric
        cnt = contours[index]
        x, y, w, h = cv.boundingRect(cnt)
    #else if no contours detected, produce error message and produce bounding box information with the entire image as the bounding box
    else: 
        print("no contour detected for image",count) 
        x = 0
        y = 0
        w, h = annotdict[imgsName[count]][:2]
        
    #produce bounding box with thickness that scales off the size of the image(for consistency)
    imgWidth, imgHeight = annotdict[imgsName[count]][:2]
    thickness = math.ceil(max(1,0.02*min(imgWidth, imgHeight)))
    cv.rectangle(img, (x,y), (x+w, y+h), (0, 255, 0), int(thickness))
    
    #append bounding box information into contour_area_ans answer list
    contour_area_ans.append([count,x,y,x+w,y+h])
    count+=1

#evaluate score of contour_area_ans_list
contour_area_ans = np.array(contour_area_ans, dtype=object)
a, p, r, f= score(contour_area_ans) 

#show image with bounding box
show(imgs_copy)

# 5) Shape-based detection based on contour perimeter

In [ ]:
#deep copy original image list
imgs_copy = copy.deepcopy(imgs)
count = 0

#produce answer list to store bounding box information(will be appended later)
contour_perimeter_ans = [];

for img in imgs_copy:
    img=cv.medianBlur(img,5,img)
    img_copy = cv.bilateralFilter(img,7,29,3)

    #canny edge detection
    canny = cv.Canny(img_copy, 20, 50, 3)

    #find contours
    contours, hierarchy = cv.findContours(canny, cv.RETR_EXTERNAL,cv.CHAIN_APPROX_SIMPLE)
    
    #if contour exist, find contour with largest area and produce bounding box information
    if(len(contours)>=1):
        maxVal = cv.arcLength(contours[0],True)
        index=0
        for i in range(len(contours)):
            metric = cv.arcLength(contours[i],True)
            if(metric>=maxVal):
                index=i
                maxVal = metric
        cnt = contours[index]
        x, y, w, h = cv.boundingRect(cnt)
    #else if no contours detected, produce error message and produce bounding box information with the entire image as the bounding box
    else: 
        print("no contour detected for image",count) 
        x = 0
        y = 0
        w, h = annotdict[imgsName[count]][:2]
    
    #produce bounding box with thickness that scales off the size of the image(for consistency)
    imgWidth, imgHeight = annotdict[imgsName[count]][:2]
    thickness = math.ceil(max(1,0.02*min(imgWidth, imgHeight)))
    cv.rectangle(img, (x,y), (x+w, y+h), (0, 255, 0), int(thickness))
    
    #append bounding box information into contour_perimeter_ans answer list
    contour_perimeter_ans.append([count,x,y,x+w,y+h])
    count+=1

#evaluate score of contour_perimeter_ans_list
contour_perimeter_ans = np.array(contour_perimeter_ans, dtype=object)
a, p, r, f= score(contour_perimeter_ans) 

#show image with bounding box
show(imgs_copy)

# 6) Compare and select the optimal bounding box for each image

In [ ]:
#deep copy original image list
imgs_copy = copy.deepcopy(imgs)

#define the list ans_list to contain the answer lists of the 3 of our previous answer lists
ans_list = []

#append each answer list to ans_list
ans_list.append(hsv_ans)
ans_list.append(contour_area_ans)
ans_list.append(contour_perimeter_ans)

#define a final_ans_list which is an answer list that holds the best bounding box information of each image(will be populated later)
final_ans_list = []
count=0

#loop through each image
for i in range(len(imgs_copy)):
    arr=[]
    
    #compile an array of bounding box answers for each image
    for ans in ans_list:
        arr.append(ans[i])    
    
    #sort each image's ans array in ascending order of aspect ratio(modified to be always above 1, invert if aspect ratio is less than 1)
    sorted_arr = sorted(arr, key=sort_func)
    
    #save result with aspect ratio closest to 1 in final ans list
    final_ans_list.append(sorted_arr[0])
    
    #generate bounding box of image
    imgWidth, imgHeight, x1, y1, x2, y2 = annotdict[imgsName[count]]
    thickness = math.ceil(max(1,0.02*min(imgWidth, imgHeight)))
    cv.rectangle(imgs_copy[count], (x1,y1), (x2,y2), (0, 255, 0), int(thickness))
    count+=1

#evaluate score of final_ans_list
final_ans_list = np.array(final_ans_list, dtype=object)
a, p, r, f = score(final_ans_list, csv=True) 

#show image with the best bounding box from the 3 provided answer lists for each method
show(imgs_copy)